In [ ]:
%matplotlib notebook 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm import tqdm 
import matplotlib.animation as anim


In [ ]:
## Definimos las ecuaciones diferenciales
##las variables corresponden a : pr, pphi, r, phi

#Valores de las constantes en Unidades internacionales
G = 6.67e-11 
mT = 5.9736e24 
rT = 6.3781e6
mL = 0.07349e24
rL = 1.7374e6
d = 3.844e8


#Constantes usadas en las ecuaciones diferenciales

omega = 2.6617e-6 #Frecuencia angular
delta = G*mT/d**3
mu = mL/mT

def rc(r,phi,t):
    return np.sqrt(1 + r**2 - 2*r*np.cos(phi - omega*t))

def f1(r, phi, pr, pphi, t):
    return pr

def f2(r, phi, pr, pphi, t):
    return pphi/r**2

def f3(r, phi, pr, pphi, t):
    return pphi**2/r**3 - delta*(1/r**2 + (mu/rc(r,phi,t)**3)*(r - np.cos(phi - omega*t)))

def f4(r, phi, pr, pphi, t):
    return -delta*mu*r/rc(r,phi,t)**3*np.sin(phi - omega*t)

#Estos momentos canónicos inciiales corresponden a los dados en el punto g)
def GetPr(v, r, phi, theta):
    return v*np.cos(theta-phi)
def GetPphi(v, r, phi, theta):
    return r*v*np.sin(theta - phi)

##Condiciones iniciales del problema

r0 = rT # La nave empieza en la tierra
v0 = 11091 #Velocidad de escape en m/s
phi0 = 0
theta0 = np.deg2rad(30)
pr0 = GetPr(v0/d, r0/d, phi0, theta0)
pphi0 = GetPphi(v0/d, r0/d, phi0, theta0)

CondIni = [r0/d, phi0, pr0, pphi0] #Condiciones Iniciales para las variables de las ecuaciones diferenciales

In [ ]:
def GetRK4(f1, f2, f3, f4, r0, t0, tf, h):
    N = int((tf + t0)/h) + 1
    t = np.linspace(t0, tf, N)
    
    #Soluciones
    r = np.zeros(N)
    phi = np.zeros(N)
    pr = np.zeros(N)
    pphi = np.zeros(N)
    
    #Asignación de las condiciones iniciales
    r[0] = r0[0]
    phi[0] = r0[1]
    pr[0] = r0[2]
    pphi[0] = r0[3]
    
    #Vectorización de Runge-Kutta de orden 4
    
    K1 = np.zeros(4)
    K2 = np.zeros(4)
    K3 = np.zeros(4)
    K4 = np.zeros(4)
    
    #RungeKutta
    for i in tqdm(range(1,N)):
        K1[0] = h*f1(r[i-1], phi[i-1], pr[i-1], pphi[i-1], t[i-1])
        K1[1] = h*f2(r[i-1], phi[i-1], pr[i-1], pphi[i-1], t[i-1]) 
        K1[2] = h*f3(r[i-1], phi[i-1], pr[i-1], pphi[i-1], t[i-1])
        K1[3] = h*f4(r[i-1], phi[i-1], pr[i-1], pphi[i-1], t[i-1])
        
        K2[0] = h*f1(r[i-1] +0.5*K1[0], phi[i-1] +0.5*K1[1] , pr[i-1] +0.5*K1[2], pphi[i-1] +0.5*K1[3], t[i-1] + h*0.5)
        K2[1] = h*f2(r[i-1] +0.5*K1[0], phi[i-1] +0.5*K1[1] , pr[i-1] +0.5*K1[2], pphi[i-1] +0.5*K1[3], t[i-1] + h*0.5)
        K2[2] = h*f3(r[i-1] +0.5*K1[0], phi[i-1] +0.5*K1[1] , pr[i-1] +0.5*K1[2], pphi[i-1] +0.5*K1[3], t[i-1] + h*0.5)
        K2[3] = h*f4(r[i-1] +0.5*K1[0], phi[i-1] +0.5*K1[1] , pr[i-1] +0.5*K1[2], pphi[i-1] +0.5*K1[3], t[i-1] + h*0.5)

        K3[0] = h*f1(r[i-1] +0.5*K2[0], phi[i-1] +0.5*K2[1] , pr[i-1] +0.5*K2[2], pphi[i-1] +0.5*K2[3], t[i-1] + h*0.5)
        K3[1] = h*f2(r[i-1] +0.5*K2[0], phi[i-1] +0.5*K2[1] , pr[i-1] +0.5*K2[2], pphi[i-1] +0.5*K2[3], t[i-1] + h*0.5)
        K3[2] = h*f3(r[i-1] +0.5*K2[0], phi[i-1] +0.5*K2[1] , pr[i-1] +0.5*K2[2], pphi[i-1] +0.5*K2[3], t[i-1] + h*0.5)
        K3[3] = h*f4(r[i-1] +0.5*K2[0], phi[i-1] +0.5*K2[1] , pr[i-1] +0.5*K2[2], pphi[i-1] +0.5*K2[3], t[i-1] + h*0.5)
        
        K4[0] = h*f1(r[i-1] + K3[0], phi[i-1] + K3[1] , pr[i-1] + K3[2], pphi[i-1] + K3[3], t[i-1] + h)
        K4[1] = h*f2(r[i-1] + K3[0], phi[i-1] + K3[1] , pr[i-1] + K3[2], pphi[i-1] + K3[3], t[i-1] + h)
        K4[2] = h*f3(r[i-1] + K3[0], phi[i-1] + K3[1] , pr[i-1] + K3[2], pphi[i-1] + K3[3], t[i-1] + h)
        K4[3] = h*f4(r[i-1] + K3[0], phi[i-1] + K3[1] , pr[i-1] + K3[2], pphi[i-1] + K3[3], t[i-1] + h)
        
        r[i] = r[i-1] +         (K1[0] + 2*K2[0] + 2*K3[0] + K4[0])/6.
        phi[i] = phi[i-1] +     (K1[1] + 2*K2[1] + 2*K3[1] + K4[1])/6.
        pr[i] = pr[i-1] +       (K1[2] + 2*K2[2] + 2*K3[2] + K4[2])/6.
        pphi[i] = pphi[i-1] +   (K1[3] + 2*K2[3] + 2*K3[3] + K4[3])/6.
        

    return r,phi,t

In [ ]:
r,phi,t = GetRK4(f1, f2, f3, f4, CondIni, 0, 7.3e5, 10)

In [ ]:
#Soluciones

#Se reduce la cantidad de datos de la simulacion

tt = []
phii = []
rr = []

for i in range(len(t)):
    if(i % 1000 == 0):
        tt+=[t[i]]
        phii+= [phi[i]]
        rr +=[r[i]]

tt= np.array(tt)
rr = np.array(rr)
phii = np.array(phii)
#Se cambia a coordenadas cartesianas para graficar
#x = np.cos(phii)*rr
#y = np.sin(phii)*rr

#xL,yL = np.cos(omega*t), np.sin(omega*t)
#plt.scatter(xL,yL)
#plt.scatter(x,y)

In [ ]:
#Solución Inciso 1 parte 2 (Animación y Respuestas)

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(1,1,1)


def init():
    ax.set_xlim(-1,1)
    ax.set_ylim(-1,1)
    ax.set_xlabel("$x \ [d]$")
    ax.set_ylabel("$y \ [d]$")
    plt.tight_layout()

def Update(i):
    
    plot = ax.clear()
    init()
    plot = ax.set_title(r'${:.2f} \ Horas, {:.2f} \ Días$'.format(tt[i]/3600, tt[i]/3600/24), fontsize=15)
    
    plot = ax.set_xlim(-1,1)
    plot = ax.set_ylim(-1,1)
    plot = ax.set_xlabel("$x \ [d]$")
    plot = ax.set_ylabel("$y \ [d]$")
    
    t = tt[i]    
    x = np.cos(phii[i])*rr[i]
    y = np.sin(phii[i])*rr[i]
    xL = np.cos(omega*t)
    yL = np.sin(omega*t)
    
    nave = plt.Circle( (x,y), 0.01 , color='k', fill=True, label="Nave")
    luna = plt.Circle( (xL,yL), 0.02 , color='red', fill=True, label="Luna")
    tierra = plt.Circle( (0,0), 0.1 , color='blue', fill=True, label="Tierra")
    
    plot = ax.add_patch(nave)
    plot = ax.add_patch(luna)
    plot = ax.add_patch(tierra)
    ax.legend()
    plt.tight_layout()
        

    return plot

Animation = anim.FuncAnimation(fig,Update,frames=len(tt),init_func=init)